# Analytics: Gutsy Returners

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import json

from matplotlib import pyplot as plt
from typing import Dict, List, Any, Callable, Optional
from tqdm.notebook import tqdm

In [ ]:
"""
Helpers for visualizing the field and plays.
"""


SIDE = 5
ENDZONE = 10
FIELD_L = 120
FIELD_W = 53.3
FIELD_MID = 60
FIELD_X = (0 - SIDE, FIELD_L + SIDE)
FIELD_Y = (0 - SIDE, FIELD_W + SIDE)
FIELD_XD = (FIELD_X[1] - FIELD_X[0]) / 10
FIELD_YD = (FIELD_Y[1] - FIELD_Y[0]) / 10
FIELD_RATIO = 1.25
FIELD_DIM = (FIELD_RATIO * FIELD_XD, FIELD_RATIO * FIELD_YD)


def plot_field(plt):
    style = {
        "c": "black",
        "alpha": 0.5,
    }
    plt.xlim(*FIELD_X)
    plt.ylim(*FIELD_Y)
    plt.plot([0, FIELD_L], [0, 0], **style)
    plt.plot([0, FIELD_L], [FIELD_W, FIELD_W], **style)
    plt.plot([0, 0], [0, FIELD_W], **style)
    plt.plot([FIELD_L, FIELD_L], [0, FIELD_W], **style)
    for x in range(ENDZONE, FIELD_L, 10):
        yard = 50 - abs(x - 10 - 50)
        no_dash = (x == FIELD_MID) or (x == ENDZONE) or (x == (FIELD_L - ENDZONE))
        style = {
            "c": "black",
            "alpha": 0.5,
            "dashes": [] if no_dash else [2, 2],
        }
        plt.plot([x, x], [0, FIELD_W], **style)
        plt.text(s=yard, x=x, y=-2, ha="center")
    fig = plt.gcf()
    fig.set_size_inches(*FIELD_DIM)
    

def team_to_color(s: str) -> str:
    return "blue" if s == "home" else "red"


def plot_play(
    df_tracking,
    event="ball_snap",
    frame=None,
    show_numbers=False
):
    if len(df_tracking) == 0:
        raise ValueError("No records in the tracking data.")

    df_track = df_tracking.copy()
    
    plot_field(plt)
    
    df_track["color"] = df_track["team"].apply(team_to_color)

    df_ball = df_track.query("team == 'football'")
    plt.scatter(x=df_ball["x"], y=df_ball["y"], c="brown", s=5)

    df_path = df_track.query("team != 'football'")
    plt.scatter(x=df_path["x"], y=df_path["y"], c=df_path["color"], s=5, alpha=0.15)

    time_query = f"event == '{event}'" if frame is None else f"frameId == {frame}"
    df_init = df_track.query(f"{time_query} and team != 'football'")
    plt.scatter(x=df_init["x"], y=df_init["y"], c=df_init["color"], s=50)
    plt.title(f"Frame {df_init['frameId'].min()} / {df_tracking['frameId'].max()}")
    
    if show_numbers:
        for p in df_init.to_dict(orient="records"):
            plt.text(s=int(p["jerseyNumber"]), x=p["x"] + 0.5, y=p["y"])

In [ ]:
MAX_DIST = 200
SIDELINE_MIN = 0
SIDELINE_MAX = 53.0 + (1.0 / 3.0)
RECEIVING_GOAL_LINE = 10
KICKING_GOAL_LINE = 110


def distance(a: Dict, b: Dict) -> float:
    return np.sqrt((a["y"] - b["y"])**2 + (a["x"] - b["x"])**2)


def closest_defender_distance(
    ball_x: float,
    ball_y: float,
    players: List[Dict],
    kickingTeam: str
) -> float:
    ball = { "x": ball_x, "y": ball_y }
    min_dist = MAX_DIST
    for p in players:
        if p["teamCode"] == kickingTeam:
            d = distance(ball, p)
            if d < min_dist:
                min_dist = d
    return min_dist


def defenders_within_radius(
    ball_x: float,
    ball_y: float,
    players: List[Dict],
    kickingTeam: str,
    radius: int
) -> int:
    ball = { "x": ball_x, "y": ball_y }
    count = 0
    for p in players:
        if p["teamCode"] == kickingTeam:
            d = distance(ball, p)
            if d < radius:
                count += 1
    return count


def blockers_within_radius(
    ball_x: float,
    ball_y: float,
    players: List[Dict],
    kickingTeam: str,
    radius: int
) -> int:
    ball = { "x": ball_x, "y": ball_y }
    count = 0
    for p in players:
        if p["teamCode"] != kickingTeam and p["x"] >= ball_x:
            d = distance(ball, p)
            if d < radius:
                count += 1
    return count


def distance_to_sideline(ball_y: float) -> float:
    d_bottom = ball_y - SIDELINE_MIN
    d_top = SIDELINE_MAX - ball_y
    # Out of bounds
    if d_bottom <= 0 or d_top <= 0:
        return 0
    # Get distance to closest sideline
    return min(d_bottom, d_top)


def speed_upfield(player: Dict) -> float:
    if pd.isna(player):
        return 0
    angle_rads = np.deg2rad(player["dir"])
    speed = player["s"]
    return speed * np.sin(angle_rads)


def speed_lateral(player: Dict) -> float:
    if pd.isna(player):
        return 0
    angle_rads = np.deg2rad(player["dir"])
    speed = player["s"]
    # Take absolute value to get lateral speed in either direction
    return speed * np.abs(np.cos(angle_rads))


def closest_defender_speed_upfield(
    ball_x: float,
    ball_y: float,
    players: List[Dict],
    kickingTeam: str
) -> float:
    ball = { "x": ball_x, "y": ball_y }
    min_dist = MAX_DIST
    closest = None
    for p in players:
        if p["teamCode"] == kickingTeam:
            d = distance(ball, p)
            if d < min_dist:
                min_dist = d
                closest = p
    return speed_upfield(closest)


def closest_defender_speed_lateral(
    ball_x: float,
    ball_y: float,
    players: List[Dict],
    kickingTeam: str
) -> float:
    ball = { "x": ball_x, "y": ball_y }
    min_dist = MAX_DIST
    closest = None
    for p in players:
        if p["teamCode"] == kickingTeam:
            d = distance(ball, p)
            if d < min_dist:
                min_dist = d
                closest = p
    return speed_lateral(closest)

In [ ]:
vec_closest_defender_distance = np.vectorize(closest_defender_distance)
vec_defenders_within_radius = np.vectorize(defenders_within_radius)
vec_blockers_within_radius = np.vectorize(blockers_within_radius)
vec_distance_to_sideline = np.vectorize(distance_to_sideline)
vec_closest_defender_speed_upfield = np.vectorize(closest_defender_speed_upfield)
vec_closest_defender_speed_lateral = np.vectorize(closest_defender_speed_lateral)

# Load Data

In [ ]:
DIR = "../input/nfl-big-data-bowl-2022"
DIR_VT = "../input/process-punt-return-decision-data"
DIR_PRED = "../input/model-training-returns-for-loss"
df_preds = pd.read_csv(f"{DIR_PRED}/predictions.csv")
df_preds["players"] = df_preds["players"].apply(lambda j: json.loads(j))
print(f"Loaded {df_preds.shape[0]:,d} plays with predictions.")
df_games = pd.read_csv(f"{DIR}/games.csv")
df_players = pd.read_csv(f"{DIR}/players.csv")
# Get patched versions from our custom output
df_plays = pd.read_csv(f"{DIR_VT}/plays_patched.csv")
df_pff = pd.read_csv(f"{DIR_VT}/pff_patched.csv")

In [ ]:
PLAY_KEYS = ["gameId", "playId"]

In [ ]:
MODEL_NAME = "lr_bal"
EXPECTED_LOSS = f"loss_{MODEL_NAME}_binary"
EXPECTED_SCORE = f"prob_{MODEL_NAME}_binary"
model_path = f"../input/model-training-returns-for-loss/models/{MODEL_NAME}_binary.joblib"
model = joblib.load(model_path)

In [ ]:
def get_returner_team(row: pd.Series) -> str:
    if row.possessionTeam == row.homeTeamAbbr:
        return row.visitorTeamAbbr
    return row.homeTeamAbbr


df_preds["returnerTeam"] = df_preds.apply(get_returner_team, axis="columns")
df_preds["returnerTeam"] = df_preds["returnerTeam"].apply(lambda t: "LV" if t == "OAK" else t)
assert df_preds["returnerTeam"].isna().sum() == 0, "Returner team column has nulls."

In [ ]:
df_preds["isGain"] = ~df_preds["isZeroOrLoss"]

In [ ]:
df_preds.columns

In [ ]:
df_preds.head()

# Muffs

In [ ]:
(
    df_preds
        [
            (df_preds.specialTeamsResult == "Muffed")
            & (df_preds.firstReturnableEvent == "punt_land")
            & (df_preds.season > 2018)
        ]
        .sort_values(by=[EXPECTED_SCORE], ascending=[False])
        [[
            *PLAY_KEYS,
            "split",
            EXPECTED_SCORE,
            "firstReturnableEvent",
            "ballYardline",
            "ballLandingYardline",
            "returnYardsGained",
            "closestDefenderDistance"
        ]]
)

In [ ]:
play_cols = [
    "quarter",
    "playDescription",
]
(
    df_preds
        .query("playId == 3010")
        .join(
            df_plays.set_index(PLAY_KEYS)[play_cols],
            on=PLAY_KEYS
        )
        [[
            *PLAY_KEYS,
            "split",
            *play_cols,
            "specialTeamsResult",
            EXPECTED_SCORE,
        ]]
)

# Gunners

In [ ]:
# df_pff["gunnerJerseys"] = df_pff["gunners"].apply(lambda s: s.split("; ") if pd.notna(s) else [])

# Gutsy Returners

In [ ]:

df_gutsy = df_preds[
    df_preds[EXPECTED_LOSS]
][[
    *PLAY_KEYS,
    "specialTeamsResult",
    "returnerNflId",
    "returnYardsGained",
    "returnerTeam",
    "returnYardsGained",
    "isGain",
    "isZeroOrLoss",
    EXPECTED_LOSS,
    EXPECTED_SCORE,
]]

In [ ]:
df_gutsy.specialTeamsResult.value_counts()

In [ ]:
df_return_teams = (
    df_preds
        .groupby("returnerNflId")
        ["returnerTeam"].unique()
        .reset_index()
)
df_return_teams["returnerTeam"] = df_return_teams["returnerTeam"].apply(lambda s: ", ".join(s))

In [ ]:
df_gutsy_returns = (
    df_gutsy
        [
            (df_gutsy.specialTeamsResult == "Return")
            | (df_gutsy.specialTeamsResult == "Muffed")
        ]
        .groupby("returnerNflId")
        ["playId"].count()
        .reset_index()
        .rename(columns={"playId": "nGutsyReturns"})
)
df_gutsy_totals = (
    df_gutsy
        .groupby("returnerNflId")
        ["playId"].count()
        .reset_index()
        .rename(columns={"playId": "nGutsy"})
)
df_gutsy_gain_rate = (
    df_gutsy
        .groupby("returnerNflId")
        ["isGain"].mean()
        .reset_index()
        .rename(columns={"isGain": "gainRateGutsy"})
)
df_gutsy_avg_yards = (
    df_gutsy
        .groupby("returnerNflId")
        ["returnYardsGained"].mean()
        .reset_index()
        .rename(columns={"returnYardsGained": "avgReturnYardsGainedOnGutsyReturns"})
)
df_gutsy_metrics = (
    df_gutsy_returns
        .join(df_gutsy_totals.set_index("returnerNflId"), on="returnerNflId")
        .join(df_gutsy_avg_yards.set_index("returnerNflId"), on="returnerNflId")
        .join(df_gutsy_gain_rate.set_index("returnerNflId"), on="returnerNflId")
)
df_gutsy_metrics["gutsyReturnRate"] = df_gutsy_metrics["nGutsyReturns"] / df_gutsy_metrics["nGutsy"]
df_gutsy_players = (
    df_gutsy_metrics
        .join(df_players.set_index("nflId"), on="returnerNflId")
        .rename(columns={"Position": "position"})
        .join(df_return_teams.set_index("returnerNflId"), on="returnerNflId")
)
(df_gutsy_players.nGutsy >= 15).sum()

In [ ]:
gutsy_show_cols = {
    "displayName": "Returner",
    "position": "Pos",
    "returnerTeam": "Teams",
    "nGutsy": "Difficult Returnable Punts Faced",
    "nGutsyReturns": "Difficult Punts Returned",
    "gutsyReturnRate": "Difficult Punt Return Rate",
    "gainRateGutsy": "Gain Rate on Difficult Returnable Punts",
    "avgReturnYardsGainedOnGutsyReturns": "Average Return Yards on Difficult Returnable Punts",
}
df_gutsy_ranks = (df_gutsy_players
    [df_gutsy_players.nGutsy >= 15]
    .sort_values(
        by=["gainRateGutsy", "nGutsy"],
        ascending=[False, False]
    )
    [gutsy_show_cols.keys()]
     .rename(columns=gutsy_show_cols)
     .reset_index(drop=True)
)
df_gutsy_ranks["Rank"] = (np.arange(len(df_gutsy_ranks)) + 1).astype(int)
(
    df_gutsy_ranks
        [["Rank"] + list(gutsy_show_cols.values())]
        .T.drop_duplicates().T
        .head(10)
        .style.format({
            "Average Return Yards on Difficult Returnable Punts": "{:.1f}",
            "Difficult Punt Return Rate": "{:.3f}",
            "Gain Rate on Difficult Returnable Punts": "{:.3f}",
        })
)

In [ ]:
line_kws = dict(
    color="black",
    alpha=0.5,
    dashes=[4, 1]
)
sns.scatterplot(
    data=df_gutsy_ranks,
    x="Difficult Punt Return Rate",
    y="Gain Rate on Difficult Returnable Punts"
)
for i, t in df_gutsy_ranks.iterrows():
    if i < 5:
        plt.text(
            s=t["Returner"],
            x=t["Difficult Punt Return Rate"],
            y=t["Gain Rate on Difficult Returnable Punts"]
        )
plt.axvline(df_gutsy_ranks["Difficult Punt Return Rate"].mean(), **line_kws)
plt.axhline(df_gutsy_ranks["Gain Rate on Difficult Returnable Punts"].mean(), **line_kws)
plt.gcf().set_size_inches(16, 8)
plt.show()

In [ ]:
df_preds["gutsyYards"] = df_preds[EXPECTED_SCORE] * df_preds["returnYardsGained"]
df_gutsy_yards = (
    df_preds
        .groupby("returnerNflId")
        ["gutsyYards"].mean()
        .reset_index()
        .rename(columns={"gutsyYards": "meanGutsyYards"})
        .join(
            df_preds
                .groupby("returnerNflId")
                ["gutsyYards"].sum()
                .reset_index()
                .rename(columns={"gutsyYards": "sumGutsyYards"})
                .set_index("returnerNflId"),
            on="returnerNflId"
        )
)

In [ ]:
result_cols = df_preds.specialTeamsResult.unique()
rate_cols = [f"{col} Rate" for col in result_cols]

df_return_results = pd.concat([
    df_preds[PLAY_KEYS + ["returnerNflId"]],
    pd.get_dummies(df_preds["specialTeamsResult"])
], axis=1)
df_return_totals = (
    df_preds
        .groupby("returnerNflId")
        ["playId"].count()
        .reset_index()
        .rename(columns={"playId": "totalReturnable"})
)
df_return_counts = (
    df_return_results
        .groupby("returnerNflId")
        [result_cols].sum()
        .reset_index()
        .join(
            df_return_totals.set_index("returnerNflId"),
            on="returnerNflId"
        )
)
df_return_freq = (
    df_return_counts
        .join(
            df_return_counts.set_index("returnerNflId")[result_cols],
            on="returnerNflId",
            rsuffix=" Rate"
        )
)
df_return_freq[rate_cols] = (
    df_return_counts
        [result_cols]
        .div(df_return_counts.totalReturnable, axis=0)
)
df_return_yards = (
    df_preds
        .groupby("returnerNflId")
        ["returnYardsGained"].mean()
        .reset_index()
        .rename(columns={"returnYardsGained": "meanReturnYards"})
        .join(
            df_gutsy
                .groupby("returnerNflId")
                ["returnYardsGained"].sum()
                .reset_index()
                .rename(columns={"returnYardsGained": "sumReturnYards"})
                .set_index("returnerNflId"),
            on="returnerNflId"
        )
)
df_return_players = (
    df_return_freq
        .join(df_players.set_index("nflId"), on="returnerNflId")
        .rename(columns={"Position": "position"})
        .join(df_return_teams.set_index("returnerNflId"), on="returnerNflId")
        .join(df_gutsy_metrics.set_index("returnerNflId"), on="returnerNflId")
        .join(df_return_yards.set_index("returnerNflId"), on="returnerNflId")
        .join(df_gutsy_yards.set_index("returnerNflId"), on="returnerNflId")
)

In [ ]:
returner_show_cols = [
    "displayName",
    "position",
    "nGutsyReturns",
    "nGutsy",
    "totalReturnable",
    "gutsyReturnRate",
    "meanGutsyYards",
    "sumGutsyYards",
    "meanReturnYards",
    "sumReturnYards",
]
(df_return_players
     .sort_values(
         by=["sumGutsyYards"],
         ascending=[False]
     )
     [returner_show_cols]
     .head())

In [ ]:
df_return_players[df_return_players.meanGutsyYards > df_return_players.meanReturnYards][returner_show_cols]

In [ ]:
df_return_players[df_return_players.returnerNflId == 45599].iloc[0]

In [ ]:
gutsy_show_cols = [
    "returnerNflId",
    "displayName",
    "position",
    "nGutsyReturns",
    "nGutsy",
    "gutsyReturnRate",
    "returnerTeam",
]
(df_return_players
    [df_return_players.nGutsy >= 15]
    .sort_values(
        by=["gutsyReturnRate", "nGutsy"],
        ascending=[False, False]
    )
    [gutsy_show_cols]
    .head(10))

# Teams

In [ ]:
dfg_team_returns = (
    df_gutsy
        [
            (df_gutsy.specialTeamsResult == "Return")
            | (df_gutsy.specialTeamsResult == "Muffed")
        ]
        .groupby("returnerTeam")
        ["playId"].count()
        .reset_index()
        .rename(columns={"playId": "nGutsyReturns"})
)
dfg_team_totals = (
    df_gutsy
        .groupby("returnerTeam")
        ["playId"].count()
        .reset_index()
        .rename(columns={"playId": "nGutsy"})
)

df_team_gain_rate = (
    df_gutsy
        .groupby("returnerTeam")
        ["isGain"].mean()
        .reset_index()
        .rename(columns={"isGain": "gainRateGutsy"})
)
df_team_avg_yards = (
    df_gutsy
        .groupby("returnerTeam")
        ["returnYardsGained"].mean()
        .reset_index()
        .rename(columns={"returnYardsGained": "avgReturnYardsGainedOnGutsyReturns"})
)
dfg_team_metrics = (
    dfg_team_returns
        .join(dfg_team_totals.set_index("returnerTeam"), on="returnerTeam")
        .join(df_team_gain_rate.set_index("returnerTeam"), on="returnerTeam")
        .join(df_team_avg_yards.set_index("returnerTeam"), on="returnerTeam")
)
dfg_team_metrics["gutsyReturnRate"] = dfg_team_metrics["nGutsyReturns"] / dfg_team_metrics["nGutsy"]

In [ ]:
team_show_cols = {
    "returnerTeam": "Team",
    "nGutsy": "Difficult Returnable Punts Faced",
    "nGutsyReturns": "Difficult Punts Returned",
    "gutsyReturnRate": "Difficult Punt Return Rate",
    "gainRateGutsy": "Gain Rate on Difficult Returnable Punts",
    "avgReturnYardsGainedOnGutsyReturns": "Average Return Yards on Difficult Returnable Punts",
}
df_team_ranks = (dfg_team_metrics
    .sort_values(
        by=["gainRateGutsy", "nGutsy"],
        ascending=[False, False]
    )
    [team_show_cols.keys()]
     .rename(columns=team_show_cols)
     .reset_index(drop=True)
)
df_team_ranks["Rank"] = (np.arange(len(df_team_ranks)) + 1).astype(int)
(
    df_team_ranks
        [["Rank"] + list(team_show_cols.values())]
)
(df_team_ranks.T.drop_duplicates().T
        .head(10)
        .style.format({
            "Average Return Yards on Difficult Returnable Punts": "{:.1f}",
            "Difficult Punt Return Rate": "{:.3f}",
            "Gain Rate on Difficult Returnable Punts": "{:.3f}",
        }))

In [ ]:
line_kws = dict(
    color="black",
    alpha=0.5,
    dashes=[4, 1]
)
sns.scatterplot(
    data=df_team_ranks,
    x="Difficult Punt Return Rate",
    y="Gain Rate on Difficult Returnable Punts"
)
for _, t in df_team_ranks.iterrows():
    plt.text(
        s=t["Team"],
        x=t["Difficult Punt Return Rate"],
        y=t["Gain Rate on Difficult Returnable Punts"]
    )
plt.axvline(df_team_ranks["Difficult Punt Return Rate"].mean(), **line_kws)
plt.axhline(df_team_ranks["Gain Rate on Difficult Returnable Punts"].mean(), **line_kws)
plt.gcf().set_size_inches(16, 8)
plt.show()


# Unexpected Returns

In [ ]:
show_cols = [
    *PLAY_KEYS,
    "split",
    "firstReturnableEvent",
    "kickingYardline",
    "receivingYardline",
    "ballLandingYardline",
    EXPECTED_LOSS,
    EXPECTED_SCORE,
    "returnYardsGained",
]
(
    df_preds
        [
            (df_preds[EXPECTED_LOSS])
            & (df_preds.returnYardsGained > 20)
        ]
        [show_cols]
        .sort_values(
            by=["returnYardsGained", EXPECTED_SCORE],
            # by=[EXPECTED_SCORE, "returnYardsGained"],
            ascending=[False, False]
        )
        .head(10)
)

In [ ]:
(
    df_preds
        [
            (df_preds[EXPECTED_LOSS])
            & (df_preds.returnYardsGained < 0)
        ]
        [show_cols]
        .sort_values(
            by=["returnYardsGained", EXPECTED_SCORE],
            # by=[EXPECTED_SCORE, "returnYardsGained"],
            ascending=[True, False]
        )
        .head(10)
)

In [ ]:
(
    df_preds
        [
            (~df_preds[EXPECTED_LOSS])
            & (df_preds.returnYardsGained > 20)
            & (df_preds.kickingYardline >= 40)
        ]
        [show_cols]
        .sort_values(
            #by=["returnYardsGained", EXPECTED_SCORE],
            by=[EXPECTED_SCORE, "returnYardsGained"],
            ascending=[True, False]
        )
        .head(10)
)

# Model Heat Map

In [ ]:
INPUT_COLS = [
    # Defender and blocker features
    "closestDefenderDistance",
    "defendersWithinRadius",
    "blockersWithinRadius",
    "closestDefenderSpeedUpfield",
    "closestDefenderSpeedLateral",
    # Field position features
    "ballYardline",
    "distanceToSideline",
    "isInsideOwnEndzone",
    "isInsideOwn20",
    "isInsideOwn10",
]

In [ ]:
ids = 2019100606, 3010
GAME_ID, PLAY_ID = ids
df_return = df_preds[(df_preds.gameId == GAME_ID) * (df_preds.playId == PLAY_ID)]
Xs = df_return[INPUT_COLS + ["possessionTeam", "players"]]
play = df_return.iloc[0]

In [ ]:
play.returnerNflId

In [ ]:
X_YDS = 120
Y_YDS = 54
HEAT_DIM = (Y_YDS, X_YDS)
X = Xs.loc[Xs.index.repeat(X_YDS * Y_YDS)].reset_index(drop=True)
row_idxs, col_idxs = np.indices(dimensions=HEAT_DIM)
X["ballX"] = col_idxs.flatten()
X["ballY"] = row_idxs.flatten()

In [ ]:
# Main inputs
bx = X["ballX"]
by = X["ballY"]
p = X["players"]
kt = X["possessionTeam"]
# Correct ball yard line and receiving yard line
RECEIVING_GOAL_LINE = 10
X["ballYardline"] = X["ballX"] - RECEIVING_GOAL_LINE
# Defender and blocker features
X["closestDefenderDistance"] = vec_closest_defender_distance(bx, by, p, kt)
X["defendersWithinRadius"] = vec_defenders_within_radius(bx, by, p, kt, 2)
X["blockersWithinRadius"] = vec_blockers_within_radius(bx, by, p, kt, 5)
# Field position features
X["distanceToSideline"] = vec_distance_to_sideline(by)
X["isInsideOwnEndzone"] = X["ballYardline"] <= 0
X["isInsideOwn20"] = X["ballYardline"] <= 20
X["isInsideOwn10"] = X["ballYardline"] <= 10
# Returner features
X["closestDefenderSpeedUpfield"] = vec_closest_defender_speed_upfield(bx, by, p, kt)
X["closestDefenderSpeedLateral"] = vec_closest_defender_speed_lateral(bx, by, p, kt)
# Remove columns used for feature engineering
X = X[INPUT_COLS]
X.shape

In [ ]:
Yp_mat = model.predict_proba(X)[:,1].reshape(*HEAT_DIM)
Yp_mat.shape

In [ ]:
plt.imshow(Yp_mat, cmap="PiYG_r")
plt.colorbar()
plot_field(plt)
for p in play["players"]:
    plt.scatter(p["x"], p["y"], color=team_to_color(p["team"]))
plt.scatter(play["ballX"], play["ballY"], color="brown")
plt.scatter(play["ballLandingX"], play["ballLandingY"], color="black")
plt.text(s="Decision", x=play["ballX"], y=play["ballY"])
plt.text(s="Landing", x=play["ballLandingX"], y=play["ballLandingY"])
plt.show()